Tutorial rdflib from [stackoverflow](https://stackoverflow.com/questions/43524943/creating-rdf-file-using-csv-file-as-input)

Further reads:

- https://github.com/KRontheWeb/csv2rdf-tutorial/blob/master/CSV2RDF-tutorial.ipynb

In [1]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD #most common namespaces
import urllib.parse #for parsing strings to URI's

In [10]:
url='https://raw.githubusercontent.com/KRontheWeb/csv2rdf-tutorial/master/example.csv'
df=pd.read_csv(url,sep=";",quotechar='"')
df.head()

,Name,Address,Place,Country,Age,Hobby,Favourite Colour
0,John,Dam 52,Amsterdam,The Netherlands,32,Fishing,Blue
1,Jenny,Leidseplein 2,Amsterdam,The Netherlands,12,Dancing,Mauve
2,Jill,52W Street 5,Amsterdam,United States of America,28,Carpentry,Cyan
3,Jake,12E Street 98,Amsterdam,United States of America,42,Ballet,Purple


In [4]:
g = Graph()
ppl = Namespace('http://example.org/people/')
loc = Namespace('http://mylocations.org/addresses/')
schema = Namespace('http://schema.org/')

In [6]:
for index, row in df.iterrows():
    g.add((URIRef(ppl+row['Name']), RDF.type, FOAF.Person))
    g.add((URIRef(ppl+row['Name']), URIRef(schema+'name'), Literal(row['Name'], datatype=XSD.string) ))
    g.add((URIRef(ppl+row['Name']), FOAF.age, Literal(row['Age'], datatype=XSD.integer) ))
    g.add((URIRef(ppl+row['Name']), URIRef(schema+'address'), Literal(row['Address'], datatype=XSD.string) ))
    g.add((URIRef(loc+urllib.parse.quote(row['Address'])), URIRef(schema+'name'), Literal(row['Address'], datatype=XSD.string) ))

In [7]:
print(g.serialize(format='turtle').decode('UTF-8'))

@prefix ns1: <http://schema.org/> .
@prefix ns2: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/Jake> a ns2:Person ;
    ns1:address "12E Street 98"^^xsd:string ;
    ns1:name "Jake"^^xsd:string ;
    ns2:age 42 .

<http://example.org/people/Jenny> a ns2:Person ;
    ns1:address "Leidseplein 2"^^xsd:string ;
    ns1:name "Jenny"^^xsd:string ;
    ns2:age 12 .

<http://example.org/people/Jill> a ns2:Person ;
    ns1:address "52W Street 5"^^xsd:string ;
    ns1:name "Jill"^^xsd:string ;
    ns2:age 28 .

<http://example.org/people/John> a ns2:Person ;
    ns1:address "Dam 52"^^xsd:string ;
    ns1:name "John"^^xsd:string ;
    ns2:age 32 .

<http://mylocations.org/addresses/12E%20Street%2098> ns1:name "12E Street 98"^^xsd:string .

<http://mylocations.org/addresses/52W%20Street%205> ns1:name "52W Street 5"^^xsd:string .

<http://mylocations.org/addresses/Dam%2052> ns1:name "Dam 52"^^xsd:string .

<http://mylocations.org/addre

In [9]:
g.serialize('output.ttl',format='turtle')